# A. データの前処理及び変数、関数の宣言

In [158]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["i like dogs", "i love coffee", "i hate milk", "you like cats", "you love milk", "you hate coffee"]
dtype = torch.float

"""
Word Processing
"""
word_list = list(set(" ".join(sentences).split()))
word_dict = {w: i for i, w in enumerate(word_list)}
number_dict = {i: w for i, w in enumerate(word_list)}
n_class = len(word_dict)


"""
TextRNN Parameter
"""
batch_size = len(sentences)
n_hidden = 5  # 

def make_batch(sentences):
    input_batch = []
    target_batch = []

    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]

        input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
        target_batch.append(target)
  
    return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)

# B. RNNの構築

In [159]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()

        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        self.Softmax = nn.Softmax(dim=1)

    def forward(self, hidden, X):
        X = X.transpose(0, 1)
        outputs, hidden = self.rnn(X, hidden)
        outputs = outputs[-1]
        model = torch.mm(outputs, self.W) + self.b
        return model

# C. 学習

In [160]:
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
    output = model(hidden, input_batch)
    loss = criterion(output, target_batch)

    if (epoch + 1) % 100 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

/Users/hoheokkim/anaconda3/lib/python3.6/site-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


Epoch: 0100 cost = 0.350794
Epoch: 0200 cost = 0.102215
Epoch: 0300 cost = 0.019897
Epoch: 0400 cost = 0.011887
Epoch: 0500 cost = 0.008380


# D. 検証

In [161]:
hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dogs', 'coffee', 'milk', 'cats', 'milk', 'coffee']


In [162]:
for parameter in model.parameters():
    print(parameter)

Parameter containing:
tensor([[-0.9252,  0.4302,  0.8146,  0.1937,  1.7612, -1.1401,  0.7427, -3.9068,
         -0.2481],
        [ 2.5777,  2.5360,  0.6304,  1.4458, -1.6813,  0.7923, -1.2673, -2.3757,
          1.3631],
        [-2.4550,  0.6091, -0.2770,  0.8030, -2.6449,  1.0009,  0.1298,  2.4856,
          0.2149],
        [-2.5399,  1.0224,  0.7613,  0.1300,  1.0377,  1.5288, -1.6626,  2.1457,
          1.1803],
        [-0.6117,  0.3769,  0.9212,  0.1197, -1.3957,  1.1589,  1.9599, -2.7997,
          0.8009]], requires_grad=True)
Parameter containing:
tensor([-0.0787,  0.1266, -0.3946, -1.9833, -1.1144, -2.2162,  0.9435,  1.2432,
        -0.6966], requires_grad=True)
Parameter containing:
tensor([[ 0.0998, -0.7603, -0.2004,  1.9505,  0.3418, -0.3627, -0.4216, -0.2708,
         -0.2881],
        [ 0.1023, -0.8960,  0.6622, -1.1051,  0.0143,  0.3702, -0.1699,  0.2059,
         -0.2099],
        [ 0.2888, -1.9985,  0.0397,  1.5349,  0.0796,  0.7711,  0.1819, -0.1057,
         -0.58

In [163]:
for parameter in model.parameters():
    print(parameter.shape)

torch.Size([5, 9])
torch.Size([9])
torch.Size([5, 9])
torch.Size([5, 5])
torch.Size([5])
torch.Size([5])
